In [1]:
# ESTUDO DO PROBLEMA DE ESCALONAMENTO DE HORÁRIOS E ALGORITMOS GENÉTICOS

# ------------------------ Problema Alvo ------------------------
# Cada programa terá vários cursos
# Cada programa terá n semestres
# Cada curso pertencerá à um semestre especifico
# Cada semestre não poderá superar mais do que 10 aulas semanais no horario

# AG       -> Programa{ Cursos, Profs, Salas } : Schedule
# Programa -> {Horario Semanal para cada o programa com n semestres}

# *OBS1: Haverá "ataques ou choque de horarios" em um schedule para programa com n semestres (ataque entre semestres)
# *OBS2: Após a geração de um schedule pra um programa, guardar restrições desse programa para aplicar em outro, caso use os mesmos requisitos (salas e professores)


# ------------------------ Algoritmo Genético ------------------------
# ___Code___
# Laço:
# - Popular
# - Fitness
# - Testar -> (Sol, S/ Sol, Evoluir)
# Fim

# Evoluir:
# - Seleção
# - Cruzamento
# - Mutação
# __________

# Conceitos:
# • Definir o que é cromossomo
# • Teste de Meta: Sim, se algum cromossomo tem fitness = 1. Não, caso o contrário.
# • Fitness: (1 - Erro/MaxErro)
#                  (Acerto/MaxAcerto)
# • Seleção: Roleta Viciada (Gera 2*n casais de cromossomos. Partição da roleta é baseada no valor Fitness)
# • Cruzamento: Cruzar casais gerados na Seleção. Ao cruzar, é pegada informação do macho e da fêmea, e são gerados 2 filhos melhores geneticamente.
# • Mutação: Mutar gene de cromossomos aleatoriamente

# ------------------------ Versões e Upgrades ------------------------
# * v1 *
# Usar apenas professores para colisão de horário  (função fitness)
# * v2 *
# Usar professores e salas para colisão de horário (função fitness)
# * v3 *
# Usar mais de um programa no algoritmo genético (upgrade de programa para departamento)

In [2]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy

In [3]:
### Data Structure for the Program Schedule Problem ###

# Macros
WEEK_SIZE         = 5
WEEK_CLASSES_SIZE = 10

# Class Teacher
class Teacher:
    _id = 0
    def __init__(self, name):
        Teacher._id  = Teacher._id +1
        self.id      = Teacher._id
        self.name    = name
        
    def __repr__(self):
        return 'Teacher {}: {}'.format(self.id, self.name)
    
# Class Classroom
class Classroom:
    _id = 0
    def __init__(self, name):
        Classroom._id = Classroom._id +1
        self.id       = Classroom._id
        self.name     = name
    
    def __repr__(self):
        return 'Classroom {}: {}'.format(self.id, self.name)
    
# Class Course
class Course:
    _id = 0
    def __init__(self, name, classes, semester, teacher = None, classroom = None):
        Course._id      = Course._id +1
        self.id         = Course._id
        self.name       = name
        self.classes    = classes
        self.ammo       = classes
        self.semester   = semester
        self.teacher    = teacher
        self.classroom  = classroom
        
    def setTeacher(self, teacher):
        self.teacher = teacher
    
    def setClassroom(self, classroom):
        self.classroom = classroom
        
    def __repr__(self):
        return '{}'.format(self.teacher.id)
        if self.teacher is not None and self.classroom is not None:
            return 'Course {}: {} ({} classes, {}o semester, {} teacher.id, {} classroom.id)'.format(self.id, self.name, self.classes, self.semester, self.teacher.id, self.classroom.id)
        elif self.teacher is not None:
            return 'Course {}: {} ({} classes, {}o semester, {} teacher.id)'.format(self.id, self.name, self.classes, self.semester, self.teacher.id)
        elif self.classroom is not None:
            return 'Course {}: {} ({} classes, {}o semester, {} classroom.id)'.format(self.id, self.name, self.classes, self.semester, self.classroom.id)
        else:
            return 'Course {}: {} ({} classes, {}o semester)'.format(self.id, self.name, self.classes, self.semester)
        
# Class Program
class Program:
    def __init__(self, name, semesters):
        self.checked        = False
        self.name           = name
        self.semesters      = semesters
        self.courses        = []
        self.teachers       = []
        self.classrooms     = []
        self.tAccu          = []
        self.cAccu          = []
        self.tCollisionMax  = 0
        self.cCollisionMax  = 0
        
    # Utils
    def getCoursesBySemester(self, semester):
        coursesSemester = []
        for i in self.courses:
            if i.semester is semester and i.ammo > 0:
                coursesSemester.append(i)
        return coursesSemester
    
    def reloadCoursesAmmo(self):
        for i in self.courses:
            i.ammo = i.classes
    
    def getTeacherByName(self, name):
        for teacher in self.teachers:
            if teacher.name == name:
                return teacher
        return None
    
    def getClassroomByName(self, name):
        for classroom in self.classrooms:
            if classroom.name == name:
                return classroom
        return None
    
    def checkCourses(self):
        for course in self.courses:
            if course.teacher is None:#or course.classroom is None:
                print('Info: Courses information are incomplete')
                return False
        print('Info: Courses information are complete')
        self.tAccu = list([0 for i in range(len(self.teachers))])
        
        for course in self.courses:
            self.tAccu[course.teacher.id-1]+=1
        
        for tAccu_ in self.tAccu:
            if tAccu_>=2:
                self.tCollisionMax += ( math.factorial(tAccu_)/(math.factorial(tAccu_-2)*2) )#Cn,2 = n!/((n-2)!
                
        return True
    
    # Add/Remove
    def addCourse(self, course):
        try:
            if course not in self.courses:
                if(sum([c.classes for c in self.getCoursesBySemester(course.semester)])+course.classes<=WEEK_CLASSES_SIZE):
                    self.courses.append(course)
        except:
            return
    
    def removeCourse(self, course):
        try:
            self.courses.remove(course)
        except:
            return
    
    def addTeacher(self, teacher):
        try:
            if teacher not in self.teachers:
                self.teachers.append(teacher)
        except:
            return
    
    def removeTeacher(self, teacher):
        try:
            self.teachers.remove(teacher)
        except:
            return
        
    def addClassroom(self, classroom):
        try:
            if classroom not in self.classrooms:
                self.classrooms.append(classroom)
        except:
            return
    
    def removeClassroom(self, classroom):
        try:
            self.classrooms.remove(classroom)
        except:
            return
        
    def __repr__(self):
        return 'Program: {} ({} semesters, {} courses, {} teachers, {} classrooms)'.format(self.name, self.semesters, len(self.courses), len(self.teachers), len(self.classrooms))

In [4]:
### --- Input Data --- ###
### Program of EngComp ###
engComp = Program('EngComp', 10)

### Teachers ###
teachers = []
teachers.append(Teacher('Joao Batista'))
teachers.append(Teacher('Joacillo'))
teachers.append(Teacher('Ricardo Guedes'))
teachers.append(Teacher('Mauricio'))
teachers.append(Teacher('Maria Eugenia'))
teachers.append(Teacher('Roberto Carlos'))
teachers.append(Teacher('Bento'))
teachers.append(Teacher('Fernando Macedo'))
teachers.append(Teacher('Marcio Andre'))
teachers.append(Teacher('Murilo'))
teachers.append(Teacher('Alisson Linhares'))
teachers.append(Teacher('Glauber'))
teachers.append(Teacher('Ernani'))
teachers.append(Teacher('George Sales'))
teachers.append(Teacher('Francisco Antonio'))
teachers.append(Teacher('Anaxagoras'))
teachers.append(Teacher('Ronaldo'))
teachers.append(Teacher('Serra'))
teachers.append(Teacher('Fernando Parente'))
teachers.append(Teacher('Francisco Jose'))
teachers.append(Teacher('Cristiane Borges'))
teachers.append(Teacher('Nidia'))
teachers.append(Teacher('Cesar Olavo'))
teachers.append(Teacher('Elias'))
teachers.append(Teacher('Carlos Alberto'))
teachers.append(Teacher('Ajalmar'))
teachers.append(Teacher('Hairon'))
teachers.append(Teacher('Eugenia Tavares'))
teachers.append(Teacher('Paulo Regis'))
teachers.append(Teacher('Cidcley'))
teachers.append(Teacher('Dijalma Freitas'))
teachers.append(Teacher('Pedrosa'))
### Put teachers on program
for teacher in teachers:
    engComp.addTeacher(teacher)
    
### Classrooms ###
classrooms = []
#classrooms.append(Classroom(''))
### Put teachers on program
for classroom in classrooms:
    engComp.addClassroom(classroom)
    
### Courses ###
courses = []
# 1 Semester
courses.append(Course('Lógica Matemática'                             ,2,1, engComp.getTeacherByName('Maria Eugenia'), engComp.getClassroomByName('')))
courses.append(Course('Introdução à Programação'                      ,3,1, engComp.getTeacherByName('Ricardo Guedes'), engComp.getClassroomByName('')))
courses.append(Course('Eletrônica Digital'                            ,3,1, engComp.getTeacherByName('Joao Batista'), engComp.getClassroomByName('')))
courses.append(Course('Cálculo I'                                     ,2,1, engComp.getTeacherByName('Roberto Carlos'), engComp.getClassroomByName('')))
# 2 Semester
courses.append(Course('Matemática Discreta'                           ,2,2, engComp.getTeacherByName('Murilo'), engComp.getClassroomByName('')))
courses.append(Course('Programação Orientada à Objetos'               ,2,2, engComp.getTeacherByName('Alisson Linhares'), engComp.getClassroomByName('')))
courses.append(Course('Eletrônica Analógica'                          ,2,2, engComp.getTeacherByName('Bento'), engComp.getClassroomByName('')))
courses.append(Course('Cálculo II'                                    ,2,2, engComp.getTeacherByName('Fernando Macedo'), engComp.getClassroomByName('')))
courses.append(Course('Física-Eletricidade'                           ,2,2, engComp.getTeacherByName('Marcio Andre'), engComp.getClassroomByName('')))
# 3 Semester
courses.append(Course('Introdução à Análise de Algoritmos'            ,2,3, engComp.getTeacherByName('Glauber'), engComp.getClassroomByName('')))
courses.append(Course('Estrutura de Dados'                            ,2,3, engComp.getTeacherByName('Ernani'), engComp.getClassroomByName('')))
courses.append(Course('Circuitos Eletrônicos'                         ,2,3, engComp.getTeacherByName('Bento'), engComp.getClassroomByName('')))
courses.append(Course('Arquitetura de Computadores'                   ,2,3, engComp.getTeacherByName('Alisson Linhares'), engComp.getClassroomByName('')))
courses.append(Course('Física-Eletromagnetismo'                       ,2,3, engComp.getTeacherByName('George Sales'), engComp.getClassroomByName('')))
# 4 Semester
courses.append(Course('Aspectos Teóricos da Computação'               ,2,4, engComp.getTeacherByName('Ernani'), engComp.getClassroomByName('')))
courses.append(Course('Pesquisa e Ordenação'                          ,2,4, engComp.getTeacherByName('Ronaldo'), engComp.getClassroomByName('')))
courses.append(Course('Microcontroladores e Microprocessadores'       ,3,4, engComp.getTeacherByName('Anaxagoras'), engComp.getClassroomByName('')))
courses.append(Course('Geometria Analítica e Algebra Linear'          ,3,4, engComp.getTeacherByName('Francisco Antonio'), engComp.getClassroomByName('')))
# 5 Semester
courses.append(Course('Metodologia Científica e Tecnológica'          ,1,5, engComp.getTeacherByName('Cristiane Borges'), engComp.getClassroomByName('')))
courses.append(Course('Cálculo Numérico'                              ,2,5, engComp.getTeacherByName('Glauber'), engComp.getClassroomByName('')))
courses.append(Course('Banco de Dados'                                ,2,5, engComp.getTeacherByName('Serra'), engComp.getClassroomByName('')))
courses.append(Course('Sistemas Lineares'                             ,2,5, engComp.getTeacherByName('Francisco Jose'), engComp.getClassroomByName('')))
courses.append(Course('Sistemas Operacionais'                         ,2,5, engComp.getTeacherByName('Fernando Parente'), engComp.getClassroomByName('')))
# 6 Semester
courses.append(Course('Engenharia de Software'                        ,2,6, engComp.getTeacherByName('Cesar Olavo'), engComp.getClassroomByName('')))
courses.append(Course('Probabilidade e Estatística'                   ,2,6, engComp.getTeacherByName('Carlos Alberto'), engComp.getClassroomByName('')))
courses.append(Course('Redes de Computadores'                         ,2,6, engComp.getTeacherByName('Nidia'), engComp.getClassroomByName('')))
courses.append(Course('Sistemas Embarcados'                           ,3,6, engComp.getTeacherByName('Elias'), engComp.getClassroomByName('')))
# 7 Semester
courses.append(Course('Interação Humano-Computador'                   ,2,7, engComp.getTeacherByName('Hairon'), engComp.getClassroomByName('')))
courses.append(Course('Computação Gráfica'                            ,2,7, engComp.getTeacherByName('Ajalmar'), engComp.getClassroomByName('')))
courses.append(Course('Grafos'                                        ,2,7, engComp.getTeacherByName('Glauber'), engComp.getClassroomByName('')))
courses.append(Course('Produção Textual'                              ,1,7, engComp.getTeacherByName('Eugenia Tavares'), engComp.getClassroomByName('')))
courses.append(Course('Introdução à Automação Industrial e Controle'  ,2,7, engComp.getTeacherByName('Joacillo'), engComp.getClassroomByName('')))
# 8 Semester
courses.append(Course('Projeto de Sistemas de Informação'             ,2,8, engComp.getTeacherByName('Hairon'), engComp.getClassroomByName('')))
courses.append(Course('Inteligência Computacional'                    ,2,8, engComp.getTeacherByName('Ronaldo'), engComp.getClassroomByName('')))
courses.append(Course('Sistemas Distribuídos'                         ,1,8, engComp.getTeacherByName('Cidcley'), engComp.getClassroomByName('')))
courses.append(Course('Sistemas de Tempo Real'                        ,1,8, engComp.getTeacherByName('Paulo Regis'), engComp.getClassroomByName('')))
courses.append(Course('Aplicações de Controle'                        ,2,8, engComp.getTeacherByName('Paulo Regis'), engComp.getClassroomByName('')))
# 9 Semester
courses.append(Course('Trabalho de Graduação Interdisciplinar'        ,1,9, engComp.getTeacherByName('Cidcley'), engComp.getClassroomByName('')))
courses.append(Course('Empreendedorismo e Gestão'                     ,1,9, engComp.getTeacherByName('Dijalma Freitas'), engComp.getClassroomByName('')))
courses.append(Course('Programação Paralela e Distribuída'            ,3,9, engComp.getTeacherByName('Cidcley'), engComp.getClassroomByName('')))
courses.append(Course('Optativa 1'                                    ,2,9, engComp.getTeacherByName('Pedrosa'), engComp.getClassroomByName('')))
# 10 Semester
courses.append(Course('Ética e Filosofia'                            ,1,10, engComp.getTeacherByName('Bento'), engComp.getClassroomByName('')))
courses.append(Course('Projeto Social'                               ,1,10, engComp.getTeacherByName('Cristiane Borges'), engComp.getClassroomByName('')))
courses.append(Course('Optativa 2'                                   ,2,10, engComp.getTeacherByName('Pedrosa'), engComp.getClassroomByName('')))
courses.append(Course('Optativa 3'                                   ,2,10, engComp.getTeacherByName('Ronaldo'), engComp.getClassroomByName('')))
### Put courses on program
for course in courses:
    engComp.addCourse(course)
    
### Print program object
print(engComp)
engComp.checkCourses()

Program: EngComp (10 semesters, 45 courses, 32 teachers, 0 classrooms)
Info: Courses information are complete


True

In [5]:
class Schedule:
    def __init__(self, program):
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        self.schedule = list([[[None for z in range((int)(WEEK_CLASSES_SIZE/WEEK_SIZE))] for j in range(WEEK_SIZE)] for i in range(program.semesters)])
        self.generateRandomSchedule(program)
        program.reloadCoursesAmmo()
        
    def __getitem__(self, key):
        return self.schedule[key]
    
    def __len__(self):
        return len(self.schedule)
    
    def generateRandomSchedule(self, program):
        for semester in range(1, program.semesters+1):
            history = []
            while(True):
                courses = program.getCoursesBySemester(semester)
                if(courses == []):
                    break
                randomCourse = random.choice(courses)
                program.courses[program.courses.index(randomCourse)].ammo-=1
                #Put class on a random place
                while(True):
                    random1 = random.randint(1,WEEK_SIZE)
                    random2 = random.randint(1,(int)(WEEK_CLASSES_SIZE/WEEK_SIZE))
                    if (random1,random2) not in history:
                        history.append((random1,random2))
                        break
                #Put random choosen course
                self.schedule[semester-1][random1-1][random2-1] = randomCourse
        return True
    
    def __repr__(self):
        strOut = '|  '
        for i in self.weekName:
            strOut += i + '  '
        strOut += '|\n'
        for i in range(len(self.schedule)):
            strOut += '|---------------------------|\n'
            if(i+1>=10):
                strOut += '|        ' + 'Semester ' + str(i+1) + '        |\n'
            else:
                strOut += '|        ' + 'Semester ' + str(i+1) + '         |\n'
            strOut += '|---------------------------|\n'
            strOut += '|  '
            for j in range(len(self.schedule[i])):
                if(self.schedule[i][j][0] is None):
                    strOut += ' ' + str(0) + '   '
                elif(self.schedule[i][j][0].id<10):
                    strOut += ' ' + str(self.schedule[i][j][0].id) + '   '
                else: 
                    strOut += ''  + str(self.schedule[i][j][0].id) + '   '
            strOut += '| AB\n'
            strOut += '|  '
            for j in range(len(self.schedule[i])):
                if(self.schedule[i][j][1] is None):
                    strOut += ' ' + str(0) + '   '
                elif(self.schedule[i][j][1].id<10):
                    strOut += ' ' + str(self.schedule[i][j][1].id) + '   '
                else: 
                    strOut += ''  + str(self.schedule[i][j][1].id) + '   '
            strOut += '| CD\n'
        return strOut

In [6]:
class GeneticAlgorithm_ProgramSchedule:
    #|------------------------------- PARAMETERS --------------------------------|
    #| maxIterations    -> Maximum amount of iterations                          |
    #| populationSize   -> Initial population size                               |
    #| geneShift        -> Genome mutation rule                                  |
    #| reproductionRate -> Rate for number of couples used at cross-over         |
    #|---------------------------------------------------------------------------|
    def __init__(self, maxIterations, populationSize, reproductionRate=0.5, geneShift=1):
        self.maxIterations     = maxIterations
        self.populationSize    = populationSize
        self.geneShift         = geneShift
        self.couplesSize       = int(self.populationSize*reproductionRate)
        self.fitnessResults    = list()
        
    def geneticRun(self, program):
        ### Initial Population
        schedulesPopulation = list([Schedule(program) for i in range(self.populationSize)])
        
        ### Iterations
        for iteration in range(self.maxIterations):
            ### Goal Test
            goalResult = self.goalTest(program, schedulesPopulation)
            if(goalResult!=-1):
                return goalResult
            
            ### Evolution (Selection, Cross-Over and Mutation)
            schedulesPopulation_new = list()
            
            ### Selection
            selectedBreeders = self.selection(schedulesPopulation)
            males   = selectedBreeders[:self.couplesSize]
            females = selectedBreeders[self.couplesSize:]
            
            ### Cross-Over
            for m,f in zip(males,females):
                s1,s2 = self.cross_over(m,f)
                schedulesPopulation_new.append(s1)
                schedulesPopulation_new.append(s2)
                
            ### Mutation
            for schedule in schedulesPopulation_new:
                #Probability of mutation happening
                if(random.randint(0,100)<10):
                    #Update individual after mutation
                    schedule = self.mutation(schedule)
                    
            ### Update population with better individuals
            schedulesPopulation = schedulesPopulation_new
        return 'Genetic Algorithm Failed!'
    
    def goalTest(self, program, schedulePopulation):
        self.fitnessResults = list(self.fitness(program,schedule) for schedule in schedulePopulation)
        print(self.fitnessResults)
        try:
            goalIndex = self.fitnessResults.index(1)
            return schedulePopulation[goalIndex]
        except(ValueError):
            return -1
    
    def fitness(self, program, schedule):
        tCollisionCount = 0.0
        # walk into week (mon-fri) and classtime (AB-CD)
        for week in range(len(schedule[0])):
            for classTime in range(len(schedule[0][0])):
                # each position to compare
                for s_out in range(len(schedule)):
                    for s_in in range(s_out+1,len(schedule)-1):
                        v_out = schedule[s_out][week][classTime]
                        v_in  = schedule[s_in][week][classTime]
                        if v_out != None and v_in != None:
                            # verify if has collision
                            if v_in.teacher.id==v_out.teacher.id:
                                # increment
                                tCollisionCount+=1
        # fitness = 1 - (collision)/(maxCollision)
        return (1 - (tCollisionCount/program.tCollisionMax))
    
    def selection(self, schedulePopulation):
        selected = []
        # select 2*couple individuals
        for i in range(2*self.couplesSize):
            ### Using Addicted Roulette to select each individual
            # sum fitness results
            fitnessResSum = sum(self.fitnessResults)
            # separate roulette portions
            roulettePortions = list(f/fitnessResSum for f in self.fitnessResults)
            # set intervals
            intervals = []
            accum = 0
            for rp in roulettePortions:
                accum+=rp
                intervals.append(round(accum,2))
            # spin addicted roulette
            rouletteResult = random.randint(0,len(schedulePopulation))/len(schedulePopulation)
            # get selected indivitual based on the addicted roulette result
            individual = 0
            while(rouletteResult>intervals[individual]):
                individual+=1
            # append on select list
            selected.append(schedulePopulation[individual])
        return selected
    
    def cross_over(self, male, female):
        # create copies of male and female
        m   = copy.deepcopy(male)
        f   = copy.deepcopy(female)
        # get a random number of cut
        cut = random.randint(0,len(m)-1)
        # cross-over male and female
        male.schedule   = m.schedule[:cut] + f.schedule[cut:]
        female.schedule = f.schedule[:cut] + m.schedule[cut:]
        return male,female
    
    def mutation(self, schedule):
        # Not Implemented yet
        return schedule

In [7]:
GA = GeneticAlgorithm_ProgramSchedule(10,4)
GA.geneticRun(engComp)

[0.7777777777777778, 0.7777777777777778, 0.8333333333333334, 0.7777777777777778]
[0.8333333333333334, 0.7777777777777778, 0.6666666666666667, 0.7777777777777778]
[0.7777777777777778, 0.7777777777777778, 0.8333333333333334, 0.8333333333333334]
[0.6666666666666667, 0.8333333333333334, 0.7777777777777778, 0.6666666666666667]
[0.8333333333333334, 0.8333333333333334, 0.6666666666666667, 0.6666666666666667]
[0.8333333333333334, 0.6666666666666667, 0.6666666666666667, 0.8333333333333334]
[0.6666666666666667, 0.9444444444444444, 0.9444444444444444, 0.9444444444444444]
[0.9444444444444444, 0.6666666666666667, 0.6666666666666667, 0.6666666666666667]
[0.6666666666666667, 0.8333333333333334, 0.6666666666666667, 0.8333333333333334]
[0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334]


'Genetic Algorithm Failed!'